 This tutorial uses the Python Google Earth
Engine API in conjunction with Google Colab, an online code editor that makes it easy to create
and share notebooks. Using Colab, we will demonstrate how to access and display satellite imagery
from Google Earth Engine. Then, we will perform some basic geoprocessing on the resulting
images. Finally, we will export the data to Google Drive.

**Before starting this tutorial, ensure that you have registered for Google Earth Engine and created a Cloud Project under this account. You will not be able to authenticate the notebook without doing so.**

## GEE Authentication and Initialization in Google Colab


In [ ]:
import ee
import geemap

In [ ]:
ee.Authenticate()
ee.Initialize(project="ee-epxdecks1")

## Import Data from Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
path = "../content/drive/MyDrive/Earth Engine Colab/rhodeisland.geojson"
region = geemap.geojson_to_ee(path)

#Displays the GeoJSON as an EEObject through geemap.
map = geemap.Map()
map.centerObject(region, 9)
map.addLayer(region, {}, 'region')
map

Map(center=[41.5940076791208, -71.52472018824878], controls=(WidgetControl(options=['position', 'transparent_b…

## Retrieve Raster Data


In [ ]:
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")

In [ ]:
centroid = region.geometry().centroid(maxError=1)
spatialFiltered = l8.filterBounds(centroid)

In [ ]:
startDate = '2015-01-01';
endDate = '2015-12-31'
temporalFiltered = spatialFiltered.filterDate(startDate, endDate);

In [ ]:
cloudless = temporalFiltered.sort('CLOUD_COVER').first();

In [ ]:
mean = temporalFiltered.reduce(ee.Reducer.mean());
median = temporalFiltered.reduce(ee.Reducer.median());
min = temporalFiltered.reduce(ee.Reducer.min());

## Displaying Rasters


In [ ]:
visParams = {'bands': ['B4', 'B3', 'B2'], 'max': 0.3};
map = geemap.Map()
map.centerObject(region, 9)
map.addLayer(cloudless, visParams, 'true-color composite');
map

Map(center=[41.5940076791208, -71.52472018824878], controls=(WidgetControl(options=['position', 'transparent_b…

## Perform Operations on Raster Data

In [ ]:
def addNDVI(image):
  ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI');
  return image.addBands(ndvi);

withNDVI = temporalFiltered.map(addNDVI)
max = withNDVI.reduce(ee.Reducer.max());

In [ ]:
map = geemap.Map()
map.centerObject(region, 9)
vizParams = {
    'bands': ['NDVI_max'], 'min': -1, 'max': 1,
    'palette': ['white', 'green']
    };
map.addLayer(max, vizParams, 'Greenest pixel composite');
map

Map(center=[41.5940076791208, -71.52472018824878], controls=(WidgetControl(options=['position', 'transparent_b…

## Export Data to Drive

In [ ]:
#region is a FeatureCollection, region.geometry is a MultiPolygon
task = ee.batch.Export.image.toDrive(
  image=max.select('NDVI_max'),
  crs='EPSG:4326',
  region=region.geometry(),
  description=f'NDVI_max_{startDate}_{endDate}',
  folder='Earth Engine Colab',
  scale=30);

task.start()

AttributeError: 'builtin_function_or_method' object has no attribute 'select'

In [ ]:
task.status()

{'state': 'RUNNING',
 'description': 'NDVI_max_2015-01-01_2015-12-31',
 'creation_timestamp_ms': 1706898540584,
 'update_timestamp_ms': 1706898543228,
 'start_timestamp_ms': 1706898543193,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'id': 'IIKKUNETTCTI3PDAJAIIRYJM',
 'name': 'projects/ee-epxdecks1/operations/IIKKUNETTCTI3PDAJAIIRYJM'}